In [1]:
!pip3 install -q -U bitsandbytes peft trl accelerate datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
model_name = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config = bnb_config, device_map = {"":0})
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [6]:
def inference(query:str,model,tokenizer) -> str:
  device = "cuda:0"

  prompt_template = '''
  bos><start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  '''

  prompt = prompt_template.format(query = query)

  model_inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [ ]:
result = inference(query="Write a bubble sort code in c++.", model=model, tokenizer=tokenizer)
print(result)

In [7]:
from datasets import load_dataset

dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
df = dataset.to_pandas()
df.head(8)

,text,output,input,instruction
0,Below is an instruction that describes a task....,# Python code\ndef sum_sequence(sequence):\n ...,"[1, 2, 3, 4, 5]",Create a function to calculate the sum of a se...
1,Below is an instruction that describes a task....,"def add_strings(str1, str2):\n """"""This func...","str1 = ""Hello ""\nstr2 = ""world""",Develop a function that will add two strings
2,Below is an instruction that describes a task....,#include <map>\n#include <string>\n\nclass Gro...,,Design a data structure in C++ to store inform...
3,Below is an instruction that describes a task....,def bubble_sort(arr):\n n = len(arr)\n \n ...,"[3, 1, 4, 5, 9, 0]",Implement a sorting algorithm to sort a given ...
4,Below is an instruction that describes a task....,import UIKit\n\nclass ExpenseViewController: U...,Not applicable,Design a Swift application for tracking expens...
5,Below is an instruction that describes a task....,<?php\n$timestamp = $_GET['timestamp'];\n\nif(...,Not Applicable,Create a REST API to convert a UNIX timestamp ...
6,Below is an instruction that describes a task....,import requests\nimport re\n\ndef crawl_websit...,website: www.example.com \ndata to crawl: phon...,Generate a Python code for crawling a website ...
7,Below is an instruction that describes a task....,"[x*x for x in [1, 2, 3, 5, 8, 13]]",,Create a Python list comprehension to get the ...


In [9]:
def generate_prompt(data_point):
  prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
  if data_point['input']:
    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
  else:
    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""

  return text

In [10]:
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt",text_column)

In [11]:
# dataset = dataset.map(generate_prompt)

In [12]:
dataset = dataset.shuffle(seed=1234)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/121959 [00:00<?, ? examples/s]

In [13]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [14]:
train_data.to_pandas().head(10)

,text,output,input,instruction,prompt,input_ids,attention_mask
0,Below is an instruction that describes a task....,create table user_accounts (\n user_id int ...,Not applicable,Create a MySQL database table for user account...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Below is an instruction that describes a task....,"def distance(x1, y1, x2, y2):\n return ((x2...",,Generate a program to calculate the distance b...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Below is an instruction that describes a task....,"<form action="""">\n <label for=""name"">Name</la...",,Construct an HTML form with three text inputs ...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Below is an instruction that describes a task....,"def get_target_sum_pair(arr, target_sum):\n ...","[1, 2, 3, 4, 5], sum = 7",Design an algorithm that takes an array of int...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Below is an instruction that describes a task....,"@app.route(""/get-data"", methods=['GET'])\ndef ...",,Create a REST API endpoint that can read data ...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,Below is an instruction that describes a task....,def capitalize_words(s):\n words_list = s.s...,"my_string = ""i love programming""","Take the following string, and capitalize all ...",<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,Below is an instruction that describes a task....,def add_one(num):\n return num + 1\n\nprint...,,Create a function that takes any number and ad...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,Below is an instruction that describes a task....,"def find_combinations(N, K):\n result = []\...","N = 5, K = 3",Generate a code for a function that takes two ...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,Below is an instruction that describes a task....,import * as InAppPurchases from 'expo-in-app-p...,Not applicable,Implement an In-App Purchase flow in React nat...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,Below is an instruction that describes a task....,def max_element(arr):\n max = arr[0]\n f...,"[5, 6, 2, 7, 1, 8]",Create an appropriate method to find the maxim...,<start_of_turn>user Below is an instruction th...,"[2, 106, 1645, 30641, 603, 671, 14239, 674, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [15]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [16]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
    

In [17]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

modules = find_all_linear_names(model)
print(modules)

['q_proj', 'k_proj', 'gate_proj', 'o_proj', 'down_proj', 'v_proj', 'up_proj']


In [22]:
lora_config = LoraConfig(
    r = 64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [23]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


In [25]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=200,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/97567 [00:00<?, ? examples/s]

Map:   0%|          | 0/24392 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [26]:
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.165100
2,3.013100
3,3.116000
4,2.534500
5,1.825800
6,1.967200
7,1.620400
8,1.557700
9,1.685100
10,1.270000


TrainOutput(global_step=200, training_loss=0.8043752980232238, metrics={'train_runtime': 2156.6284, 'train_samples_per_second': 0.742, 'train_steps_per_second': 0.093, 'total_flos': 4964227461267456.0, 'train_loss': 0.8043752980232238, 'epoch': 0.02})

In [27]:
fine_tuned_model = "gemma-2b-it-code-finetuned"

In [28]:
trainer.model.save_pretrained(fine_tuned_model)

merge the lora configured model with the base model and save the model and tokenizer to push it to the hub

In [29]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, fine_tuned_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [32]:
merged_model.push_to_hub(fine_tuned_model, use_temp_dir=False)
tokenizer.push_to_hub(fine_tuned_model, use_temp_dir=False)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/omparghale/gemma-2b-it-code-finetuned/commit/102cc5f5e0bf3d3dbfc1828763054b989ef26b58', commit_message='Upload tokenizer', commit_description='', oid='102cc5f5e0bf3d3dbfc1828763054b989ef26b58', pr_url=None, pr_revision=None, pr_num=None)

# **Test the fine-tuned model:**


**Task 1:**<br>Write a c++ code for fibonacci sequence upto input n using recursion.<br><br>
**Task 2:**<br>Your task is to minimize the function via Gradient Descent: f(x)=x^2.Write code in python. sample function declaration: def get_minimizer(self, iterations: int, learning_rate: float, init: int)

In [39]:
gen_code = inference(query = '''''',model=merged_model,tokenizer=tokenizer)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [40]:
print(gen_code)


  bos>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  Your task is to minimize the function via Gradient Descent: f(x)=x^2.Write code in python. sample function declaration: def get_minimizer(self, iterations: int, learning_rate: float, init: int)
  
model


  ```python
import numpy as np

def get_minimizer(self, iterations: int, learning_rate: float, init: int) -> None:
    """
    This function minimizes the function f(x) = x^2 using gradient descent.

    Args:
        iterations (int): Number of iterations to run the gradient descent algorithm.
        learning_rate (float): Learning rate for the gradient descent algorithm.
        init (int): Index of the initial point to start gradient descent from.

    Returns:
        None
    """

    # Set up the initial gradient descent step size
    step_size = 0.01

    # Initialize the gradient and set the initial position
    gradient = np.zeros_like(init)
    position 